# 3. Modules & Losses

In [1]:
import torch
import torch.nn as nn

## An Example

In [2]:
conv_1d = torch.nn.Conv1d(in_channels=5, out_channels=2, kernel_size=3)
print(conv_1d)

Conv1d(5, 2, kernel_size=(3,), stride=(1,))


In [3]:
print(conv_1d.weight)

Parameter containing:
tensor([[[ 0.2154,  0.2204,  0.0350],
         [-0.0665,  0.1452,  0.1536],
         [-0.2178,  0.0894, -0.0414],
         [ 0.0771,  0.0162, -0.0879],
         [-0.1109,  0.0510, -0.1648]],

        [[ 0.0353,  0.1956,  0.0603],
         [ 0.1143,  0.0364, -0.0381],
         [ 0.0462, -0.0757,  0.0315],
         [-0.0611,  0.0320,  0.2139],
         [ 0.1690,  0.1500, -0.2123]]], requires_grad=True)


In [4]:
for name, tensor in conv_1d.named_parameters():
    print("{:6s}  -  {}".format(name, tensor.shape))

weight  -  torch.Size([2, 5, 3])
bias    -  torch.Size([2])


---
## Basic Usage

In [5]:
# Each instance of a module has its own parameters initialised randomly

linear_regression_model = torch.nn.Linear(in_features=5, out_features=2)

linear_regression_model.weight

Parameter containing:
tensor([[-0.3687, -0.1563,  0.2723, -0.1297, -0.4112],
        [ 0.2871, -0.1857, -0.4047,  0.2776,  0.3073]], requires_grad=True)

In [6]:
type(linear_regression_model.weight)

torch.nn.parameter.Parameter

A `Parameter` is a Tensor which is automatically added to the list of parameters when used within a model.

In [7]:
# Pytorch Modules operate on batches. It allows to process multiple datapoints in parallel
batch_size = 3

x = torch.randn(batch_size, 5) # batch of 3 samples with 5 features each

print(x, "\n\n", x.shape)

tensor([[ 0.0071,  0.5690, -0.4557, -1.8440,  0.9468],
        [ 1.0385, -0.6421, -0.9227,  1.2484,  0.5845],
        [-0.6613,  1.4026,  2.2341,  1.0234,  0.0652]]) 

 torch.Size([3, 5])


In [8]:
predicted_y = linear_regression_model(x)  # Note, you do not call forward
print(predicted_y, "\n\n", predicted_y.shape)

tensor([[-0.4542,  0.2697],
        [-1.0243,  1.7270],
        [ 0.3853, -0.6404]], grad_fn=<AddmmBackward>) 

 torch.Size([3, 2])


---

## Composing modules with `torch.nn.Sequential`


In [9]:
neural_net = torch.nn.Sequential(
    torch.nn.Linear(5, 10),
    torch.nn.ReLU(),
    torch.nn.Linear(10, 2),
)

# Run the model:
neural_net(x)

tensor([[-0.1530,  0.2375],
        [-0.2578,  0.1088],
        [-0.7079,  0.6241]], grad_fn=<AddmmBackward>)

---

## Building Custom modules

In [10]:
import torch.nn.functional as F

class MyNeuralNetwork(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MyNeuralNetwork, self).__init__()
        
        self.linear_1 = torch.nn.Linear(input_size, hidden_size)
        self.linear_2 = torch.nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = F.relu(self.linear_1(x))
        out = self.linear_2(out)
        return out

In [11]:
x = torch.rand(5, 10)  # the first dimension is reserved for the 'batch_size'

model = MyNeuralNetwork(input_size=10, hidden_size=5,  num_classes=2)

out = model(x)  # this calls model.forward(x)

print(out.shape)

torch.Size([5, 2])


---

## Moving your model to GPU

In [12]:
model.cuda()  # No need to assign it

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

In [ ]:
device = torch.device("cuda:0")
model.to(device)  # No need to assign it

---

## Storing and loading models

### The easy way

In [13]:
torch.save(model, "my_model.pt")

/home/evann/.local/lib/python3.6/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type MyNeuralNetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [14]:
my_model_loaded = torch.load("my_model.pt")

In [15]:
print(model.linear_2.bias)
print(my_model_loaded.linear_2.bias)

Parameter containing:
tensor([ 0.2229, -0.1761], requires_grad=True)
Parameter containing:
tensor([ 0.2229, -0.1761], requires_grad=True)


### The recommended way

In [16]:
torch.save(model.state_dict(), "my_model_state_dict.pt")

In [17]:
my_model_loaded = MyNeuralNetwork(10, 5, 2)
my_model_loaded.load_state_dict(torch.load("my_model_state_dict.pt"))

<All keys matched successfully>

In [18]:
print(model.linear_2.bias)
print(my_model_loaded.linear_2.bias)

Parameter containing:
tensor([ 0.2229, -0.1761], requires_grad=True)
Parameter containing:
tensor([ 0.2229, -0.1761], requires_grad=True)


---
## Losses

---
# Building our training loop (3 / 5)

In [19]:
# INITIALIZATION

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, ToTensor, RandomCrop
from torchvision.datasets import ImageFolder

device = torch.device("cpu")

transform = Compose((RandomCrop((50, 50)), ToTensor()))
dataset = ImageFolder(root="../alien-vs-predator/", transform=transform)
loader = DataLoader(dataset, batch_size=5, shuffle=True)

model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(7500, 100),
    torch.nn.ReLU(),
    torch.nn.Linear(100, 2),
)
model.to(device)

loss_fn = nn.CrossEntropyLoss()

In [20]:
# TRAINING LOOP

for samples, labels in loader:
    samples = samples.to(device)
    labels = labels.to(device)
    predictions = model(samples)
    loss = loss_fn(predictions, labels)
    # compute gradients
    # update model parameters